### 🐍環境変数に.jsonファイルの場所を追加している。

In [ ]:
import os
import io
from PIL import Image, ImageDraw, ImageFont
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "***/***/***.json"

### 🐍矩形を描画する
- 引数：描画ファイル(pil.ImageDraw()のやつ)、左上の座標と右下の座標のタプル、矩形の色、矩形の太さ
- pillowには、矩形の線分の太さを変更するオプションがなかったため、任意の太さで描画できるように定義した。
- 太さ1の矩形をwidthの回数分だけ描画している。
- ソースは[ここ](https://stackoverflow.com/questions/34255938/is-there-a-way-to-specify-the-width-of-a-rectangle-in-pil)

In [ ]:
def draw_rectangle(draw, coordinates, color, width=1):
    for i in range(width):
        rect_start = (coordinates[0][0] - i, coordinates[0][1] - i)
        rect_end = (coordinates[1][0] + i, coordinates[1][1] + i)
        draw.rectangle((rect_start, rect_end), outline = color)
        
    return

### 🐍物体の位置と種類を取得
- 引数：画像ファイル名
- 返り値：objects(物体の名前、位置情報、確信度を含む)

In [ ]:
def localize_objects(file_path):

    from google.cloud import vision_v1p3beta1 as vision
    client = vision.ImageAnnotatorClient()

    with open(file_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations
    
    return objects

### 🐍物体を囲む矩形と、物体名のテキストを描画する
- 引数：localize_objectにより返されるobjects、画像ファイル名、出力ファイル名（オプション）
- 返り値：なし
- 出力：画像ファイル

In [ ]:
def draw_objects(objects, file_path, savefile_path='Output.jpg'):
    
    img = Image.open(file_path)
    draw = ImageDraw.Draw(img)
    fnt = ImageFont.truetype('resource/yasashisa.ttf',60)
    
    for object_ in objects:
        vects_normalized = object_.bounding_poly.normalized_vertices
        vects = ((vects_normalized[0].x * img.size[0], vects_normalized[0].y * img.size[1]),(vects_normalized[2].x * img.size[0], vects_normalized[2].y * img.size[1]))
        outline_width = 10
        outline_color = 'red'
        
        draw_rectangle(draw, vects, outline_color, outline_width)
        
        left_bottom_coordinate = (vects_normalized[0].x * img.size[0], vects_normalized[2].y * img.size[1])
        
        draw.text(left_bottom_coordinate,'物体：{}'.format(object_.name) , fill=(255, 0, 0), font=fnt)

    
    img.save(savefile_path, 'JPEG')
    print('Saved ' + savefile_path)

### 🐍以下、実行部分

In [ ]:
file_path = 'resource/objects.jpg'

In [ ]:
objects = localize_objects(file_path)

In [ ]:
draw_objects(objects, file_path)